# Spam Detector

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Retrieve the Data

The data is located at [https://static.bc-edx.com/ai/ail-v-1-0/m13/challenge/spam-data.csv](https://static.bc-edx.com/ai/ail-v-1-0/m13/challenge/spam-data.csv)

Dataset Source: [UCI Machine Learning Library](https://archive.ics.uci.edu/dataset/94/spambase)

Import the data using Pandas. Display the resulting DataFrame to confirm the import was successful.

In [4]:
# Import the data
data = pd.read_csv("https://static.bc-edx.com/ai/ail-v-1-0/m13/challenge/spam-data.csv")
data.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4601 entries, 0 to 4600
Data columns (total 58 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   word_freq_make              4601 non-null   float64
 1   word_freq_address           4601 non-null   float64
 2   word_freq_all               4601 non-null   float64
 3   word_freq_3d                4601 non-null   float64
 4   word_freq_our               4601 non-null   float64
 5   word_freq_over              4601 non-null   float64
 6   word_freq_remove            4601 non-null   float64
 7   word_freq_internet          4601 non-null   float64
 8   word_freq_order             4601 non-null   float64
 9   word_freq_mail              4601 non-null   float64
 10  word_freq_receive           4601 non-null   float64
 11  word_freq_will              4601 non-null   float64
 12  word_freq_people            4601 non-null   float64
 13  word_freq_report            4601 

## Predict Model Performance

You will be creating and comparing two models on this data: a Logistic Regression, and a Random Forests Classifier. Before you create, fit, and score the models, make a prediction as to which model you think will perform better. You do not need to be correct! 

Write down your prediction in the designated cells in your Jupyter Notebook, and provide justification for your educated guess.

I am predicting that the Random Forest Classifier will preform better because 
1. There is only numeric data.
2. There are 58 columns and decision tree algorithms (such as Random Forest Classifier) handle high-dimensional data better than Logistic Regression.
3. There are 4601 rows. That should be sufficient to overcome the danger of decision trees overfitting on small datasets.
4. Logistic Regression performs better when there is a linear relationship between features and outcome. With 57 features it is unlikely that all those features have a linear relationship to the outcome.
5. Logistic Regression is prone to be skewed by outliers. It is unlikely that there would not be any outliers with the large number of features.

Source: Willig, Gustav (Jan 16, 2023). *Decision Tree vs Logistic Regression*, https://gustavwillig.medium.com/decision-tree-vs-logistic-regression-1a40c58307d0#:~:text=Decision%20trees%20are%20often%20better,features%20and%20the%20target%20variable.

## Split the Data into Training and Testing Sets

In [7]:
# Create the labels set `y` and features DataFrame `X`
y = data['spam']
X = data.drop(columns='spam')

display(X.head())
display(y[:5])

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191


0    1
1    1
2    1
3    1
4    1
Name: spam, dtype: int64

In [8]:
# Check the balance of the labels variable (`y`) by using the `value_counts` function.
y.value_counts()/len(y)

spam
0    0.605955
1    0.394045
Name: count, dtype: float64

The dataset contains about 60% legitimate emails and about 40% spam emails. Therefore, this dataset is fairly balanced. 

In [9]:
# Split the data into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Scale the Features

Use the `StandardScaler` to scale the features data. Remember that only `X_train` and `X_test` DataFrames should be scaled.

In [10]:
from sklearn.preprocessing import StandardScaler

# Create the StandardScaler instance
scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
scaler.fit(X_train)

StandardScaler()

In [12]:
# Scale the training data
X_train_scaled = scaler.transform(X_train)

# Preview the scaled train data
X_train_scaled

array([[-0.33933373, -0.16214839, -0.55144324, ..., -0.11649103,
        -0.21175305, -0.41909144],
       [ 1.8777157 , -0.16214839, -0.55144324, ..., -0.10814021,
        -0.18841316, -0.35960578],
       [-0.33933373, -0.16214839,  2.2325961 , ...,  0.21993678,
         0.29705664, -0.10941609],
       ...,
       [-0.33933373, -0.16214839, -0.55144324, ..., -0.08642808,
        -0.20241709, -0.40334524],
       [-0.33933373,  0.02143809, -0.55144324, ..., -0.10902256,
        -0.18841316, -0.18639753],
       [-0.00843083, -0.16214839,  0.25239911, ..., -0.09960031,
        -0.1137255 ,  0.10403246]])

In [14]:
# Scale the test data
X_test_scaled = scaler.transform(X_test)

# Preview the scaled test data
X_test_scaled

array([[-0.33933373, -0.16214839, -0.55144324, ..., -0.12843428,
        -0.2444289 , -0.48382584],
       [ 2.01007686, -0.16214839,  0.84057643, ..., -0.12742588,
        -0.23976092, -0.43308807],
       [-0.33933373, -0.16214839,  1.23269465, ..., -0.11835027,
        -0.21642103, -0.30886801],
       ...,
       [-0.1077017 ,  0.01345607,  1.05624145, ..., -0.05189034,
         0.09633355,  1.21501465],
       [-0.33933373, -0.16214839, -0.55144324, ..., -0.10952676,
        -0.23042497, -0.47507794],
       [-0.33933373, -0.16214839, -0.23774866, ..., -0.07788818,
        -0.11839347,  0.83710575]])

## Create and Fit a Logistic Regression Model

Create a Logistic Regression model, fit it to the training data, make predictions with the testing data, and print the model's accuracy score. You may choose any starting settings you like. 

In [30]:
# Train a Logistic Regression model and print the model score
from sklearn.linear_model import LogisticRegression

# Create a Logistic Regression model instance
lr_model = LogisticRegression(random_state=1)

# Fit the model to the train data
lr_model.fit(X_train_scaled, y_train)

# Print the model scores for train and test data 
print(f"Logistic Regression Train Accuracy: {lr_model.score(X_train_scaled, y_train):.3f}.")
print(f"Logistic Regression Test Accuracy: {lr_model.score(X_test_scaled, y_test):.3f}.")

Logistic Regression Train Accuracy: 0.926.
Logistic Regression Test Accuracy: 0.923.


Train and test accuracy are almost identical. Therefore, the model does not seem to be overfit.

In [31]:
# Make and save testing predictions with the saved logistic regression model using the test data
lr_test_predictions = lr_model.predict(X_test_scaled)
# Review the predictions
lr_test_predictions

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [27]:
# Calculate the accuracy score by evaluating `y_test` vs. `lr_test_predictions`.
print(f"Logistic Regression Accuracy Score for Test Data: {accuracy_score(y_test, lr_test_predictions):.3f}.")

Logistic Regression Accuracy Score for Test Data: 0.923.


Not surprisingly, this is identical to the model score for the test data we printed above. 

## Create and Fit a Random Forest Classifier Model

Create a Random Forest Classifier model, fit it to the training data, make predictions with the testing data, and print the model's accuracy score. You may choose any starting settings you like. 

In [32]:
# Train a Random Forest Classifier model and print the model score
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest Classifier model instance
rf_model = RandomForestClassifier(random_state=1)

# Fit the model to the train data
rf_model.fit(X_train_scaled, y_train)

# Print the model scores for train and test data
print(f"Random Forest Classifier Train Accuracy: {rf_model.score(X_train_scaled, y_train):.3f}.")
print(f"Random Forest Classifier Test Accuracy: {rf_model.score(X_test_scaled, y_test):.3f}.")

Random Forest Classifier Train Accuracy: 0.999.
Random Forest Classifier Test Accuracy: 0.958.


Again, accuracy scores for train and test datasets are close enough to each other that it appears unlikely that the model is overfit.

In [33]:
# Make and save testing predictions with the saved Random Forest Classifier model using the test data
rf_test_predictions = rf_model.predict(X_test_scaled)

# Review the predictions
rf_test_predictions

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [34]:
# Calculate the accuracy score by evaluating `y_test` vs. `rf_test_predictions`.
print(f"Random Forest Classifier Accuracy Score for Test Data: {accuracy_score(y_test, rf_test_predictions):.3f}.")

Random Forest Classifier Accuracy Score for Test Data: 0.958.


Not surprisingly, this is identical to the model score for the test data we printed above. 

## Evaluate the Models

Which model performed better? How does that compare to your prediction? Write down your results and thoughts in the following markdown cell.

As expected, the Random Forest Classifier model performed better with an accuracy sore of 95.8% on the test data as opposed to the Logistic Regression model which had an accuracy score of 92.3% on the test data.